In [20]:
import os
import warnings
import pandas as pd 
import numpy as np
import datetime as dt

import seaborn as sns
sns.set_style('whitegrid')
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)

from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler


warnings.filterwarnings('ignore')
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [21]:
#creation dataframe

df1 = pd.read_csv('data.csv',encoding = 'unicode_escape')

In [22]:
# calculation total price 
df=df1.copy()
df["total_price"]=df["Quantity"]*df["UnitPrice"]

# Analyse RFM

RFM est l'abréviation de **Recency (récence)**, **Frequency (fréquence)** et **Monetary value (valeur monétaire)**, chacun correspondant à une caractéristique clé du client. Ces mesures RFM sont des indicateurs importants du comportement d'un client, car la fréquence et la valeur monétaire ont une incidence sur la valeur à vie d'un client, et la récence a une incidence sur la rétention, une mesure de l'engagement.

Les facteurs RFM illustrent ces faits :

Plus l'achat est récent, plus le client est réactif aux promotions

Plus le client achète fréquemment, plus il est engagé et satisfait.

La valeur monétaire différencie les gros consommateurs des acheteurs à faible valeur ajoutée.

In [23]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
print(df['InvoiceDate'].max())

2011-12-09 12:50:00


In [24]:
today_date = dt.datetime(2011,12,11)

rfm = df.groupby('CustomerID').agg({'InvoiceDate': lambda invoice_date: (today_date - invoice_date.max()).days,
                                    'InvoiceNo': lambda invoice: invoice.nunique(),
                                    'total_price': lambda total_price: total_price.sum()})

rfm.columns = ['recency','frequency','monetary']
rfm['monetary'] = rfm[rfm['monetary'] > 0]
rfm = rfm.reset_index()
rfm.head()

,CustomerID,recency,frequency,monetary
0,12346.0,326,2,NaN
1,12347.0,3,7,3.0
2,12348.0,76,4,76.0
3,12349.0,19,1,19.0
4,12350.0,311,1,311.0


# RFM Scores

In [25]:
rfm['recency_score'] = pd.qcut(rfm['recency'],5,labels=[5,4,3,2,1])

rfm['frequency_score'] = pd.qcut(rfm['frequency'].rank(method="first"),5,labels=[1,2,3,4,5])

rfm['monetary_score'] = pd.qcut(rfm['monetary'],5,labels=[1,2,3,4,5])

rfm['RFM_SCORE'] = (rfm['recency_score'].astype(str)+ rfm['frequency_score'].astype(str))

# Segmentation

In [26]:
seg_map = {r'[1-2][1-2]': 'hibernating',
           r'[1-2][3-4]': 'at_Risk',
           r'[1-2]5': 'cant_loose',
           r'3[1-2]': 'about_to_sleep',
           r'33': 'need_attention',
           r'[3-4][4-5]': 'loyal_customers',
           r'41': 'promising',
           r'51': 'new_customers',
           r'[4-5][2-3]': 'potential_loyalists',
           r'5[4-5]': 'champions'}

rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
rfm.head()

,CustomerID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
0,12346.0,326,2,NaN,1,2,NaN,12,hibernating
1,12347.0,3,7,3.0,5,4,1,54,champions
2,12348.0,76,4,76.0,2,3,4,23,at_Risk
3,12349.0,19,1,19.0,4,1,2,41,promising
4,12350.0,311,1,311.0,1,1,5,11,hibernating


# Segment Analysis

In [27]:
rfm[['recency','monetary','frequency','segment']].groupby('segment').agg({'mean','std','max','min'})

recency                            monetary             \
                        max        std        mean min      max        std   
segment                                                                      
about_to_sleep           72  11.054978   53.046070  33     72.0  11.092365   
at_Risk                 373  72.616301  158.653400  73    373.0  72.161365   
cant_loose              313  67.255524  134.088235  73    313.0  67.255524   
champions                12   3.449341    6.075188   1     12.0   3.449341   
hibernating             374  92.027549  218.221273  73    374.0  90.850537   
loyal_customers          72  15.991633   33.289308  13     72.0  15.989333   
need_attention           72  11.602720   49.726316  33     72.0  11.657708   
new_customers            12   3.701084    7.238095   1     12.0   3.701084   
potential_loyalists      32   9.304220   16.571429   1     32.0   9.310047   
promising                32   5.333629   23.103093  13     32.0   5.296101   

                                      frequency                            
                           mean   min       max        std       mean min  
segment                                                                    
about_to_sleep        52.972527  33.0         2   0.454366   1.289973   1  
at_Risk              157.800336  73.0         7   1.219826   3.313433   2  
cant_loose           134.088235  73.0        35   4.298355   9.823529   7  
champions              6.075188   1.0       248  19.531051  14.590977   4  
hibernating          215.483333  73.0         2   0.401890   1.202279   1  
loyal_customers       33.267003  13.0        76   5.761303   7.991195   4  
need_attention        49.765957  33.0         4   0.493031   2.636842   2  
new_customers          7.238095   1.0         1   0.000000   1.000000   1  
potential_loyalists   16.574949   1.0         4   0.684430   2.216327   1  
promising             23.187500  13.0         1   0.000000   1.000000   1

In [28]:
data = go.Bar(y = pd.DataFrame(rfm['segment'].value_counts()).index,
              x = pd.DataFrame(rfm['segment'].value_counts())['segment'].values,
              name = str(pd.DataFrame(rfm['segment'].value_counts())['segment'].values),
              orientation='h')

layout = go.Layout(title={'text': "Number of Customer by Segments",
                          'y':0.9,
                          'x':0.5,
                          'xanchor': 'center',
                          'yanchor': 'top'},
                   xaxis =dict(title='Customers'),
                   template = 'plotly_white')

fig=go.Figure(data=data, layout=layout)
fig.update_xaxes(range=[0,1300])
iplot(fig)


<Figure size 432x288 with 0 Axes>

In [29]:
fig = go.Figure()
for i in rfm['segment'].unique():
    fig.add_trace(go.Scatter(x=rfm[rfm['segment']==i]['recency'],
                           y=rfm[rfm['segment']==i]['frequency'],
                           mode='markers', name=str(i),showlegend = True,
                           marker = dict(size = 10,opacity = 0.6)))

fig.update_layout(title={'text': "Recency & Frequency by Segments",
                         'y':0.9,
                         'x':0.5,
                         'xanchor': 'center',
                         'yanchor': 'top'},
                  legend=dict(x=0.8,
                              y=1,
                              bgcolor='rgba(255, 255, 255, 0)'),
                  xaxis = dict(title='Recency'),
                  yaxis=dict(title='Frequency'),
                  template='plotly_white')

fig.show()



<Figure size 432x288 with 0 Axes>

# Conclusion

Le premier groupe est le groupe  et **champions** qui représente les meilleurs clients. Le traitement n'est pas nécessaire car ils sont très engagés - ils font souvent des achats, les font beaucoup et dépensent beaucoup d'argent. Cependant, il convient d'envisager des outils pour les fidéliser. La stratégie marketing pourrait ici consister à leur proposer des offres personnalisées (par exemple, des coupons d'argent) en fonction de leur activité antérieure (achats). Des remises supplémentaires peuvent ne pas être nécessaires pour ce groupe en raison de leur implication déjà élevée.

**Loyal_customers**-Ces clients sont fidèles car ils effectuent fréquemment des achats. Leur valeur monétaire peut être augmentée en leur proposant des programmes de fidélisation (remises personnalisées, cash back, livraison gratuite) ou, dans le cas de petits investissements, des bulletins d'information promotionnels.

Les labels **potencial_loyalist** . Ces clients sont prometteurs. Bien qu'ils fassent des achats fréquents, il est possible de les motiver à dépenser davantage. La stratégie la moins coûteuse peut consister à envoyer des rappels - des notifications par e-mail, par exemple, lorsque des articles se trouvent dans le panier mais que le client ne les a pas achetés, ou à rappeler au client ses récentes recherches. Des offres spéciales peuvent également être envisagées ici pour augmenter l'engagement et donc la valeur monétaire.

**cant_loose** et **new_customers**. Clients débutants. Les e-mails déclenchés peuvent être appliqués comme stratégie pour motiver les clients à faire de nouveaux achats. Un programme de fidélisation sera un avantage.
Plus tôt ils feront partie d'un tel programme, plus il y aura de possibilités de les engager avec des offres spéciales et plus vite d'autres approches d'engagement pourront être mises en œuvre.

**about to sleep** et **at risk** Ces clients sont perdus. Ils ont passé quelques commandes et n'ont pas été actifs récemment.